# Entrenamiento de un Modelo de spaCy para Detección de Entidades Nombradas (NER)
---
Este notebook tiene como objetivo entrenar un modelo de procesamiento de lenguaje natural (NLP) utilizando la biblioteca spaCy para realizar tareas de detección de entidades nombradas (NER, por sus siglas en inglés).

La detección de entidades nombradas es una técnica clave en NLP que permite identificar y clasificar entidades específicas en un texto, como nombres de personas, organizaciones, ubicaciones, fechas, cantidades, entre otras.

## Objetivos del Notebook

1. **Preparar los datos de entrenamiento**: Generar un conjunto de datos etiquetados para el entrenamiento del modelo.
2. **Configurar el modelo**: Definir los parámetros del modelo y configurar spaCy para trabajar con datos personalizados.
3. **Entrenar el modelo**: Utilizar los datos para ajustar el modelo de spaCy y mejorar su capacidad para detectar las entidades deseadas.
4. **Evaluar el modelo**: Validar el rendimiento del modelo utilizando métricas como precisión, exhaustividad y F1.
5. **Guardar y reutilizar el modelo**: Exportar el modelo entrenado para integrarlo en otras aplicaciones.



---

*__Importación de librerías__*

In [1]:
import pandas as pd
import random

import pandas as pd
from sklearn.model_selection import train_test_split
import spacy

import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example
from pathlib import Path
from spacy.tokens import Doc, Span, Token
from spacy.scorer import Scorer

# 1. Preparar los datos de entrenamiento

### 1.1 Obtener ejemplos de calles

Para crear nuestro set de entrenamiento, utilizaremos las calles de la ciudad de Madrid. Así, crearemos ejemplos con nombres reales que más tarde ayudarán a generalizar al modelo.

Descargamos los datos del callejero de Madrid desde [aquí](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=b3c41f3cf6a6c410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default)


In [2]:
df_callejero = pd.read_csv('../data/VialesVigentes_20241226.csv', sep = ';', encoding = 'cp1252')

Una vez cargados los datos, los visualizamos para entender con qué información estamos trabajando. Nos encontramos con 15 columnas, de las cuales solo nos interesan 3: VIA_CLASE, VIA_PAR y VIA_NOMBRE.

In [3]:
df_callejero.head(5)

,COD_VIA,VIA_CLASE,VIA_PAR,VIA_NOMBRE,VIA_NOMBRE_ACENTOS,COD_VIA_COMIENZA,CLASE_COMIENZA,PARTICULA_COMIENZA,NOMBRE_COMIENZA,NOMBRE_ACENTOS_COMIENZA,COD_VIA_TERMINA,CLASE_TERMINA,PARTICULA_TERMINA,NOMBRE_TERMINA,NOMBRE_ACENTOS_TERMINA
0,31001337,AUTOVÍA,NaN,A-1,A-1,31001349,AUTOVÍA,NaN,M-30,M-30,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
1,31001336,AUTOVÍA,NaN,A-2,A-2,310200,CALLE,DE,FRANCISCO SILVELA,FRANCISCO SILVELA,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
2,31001342,AUTOVÍA,NaN,A-3,A-3,480800,PLAZA,DE,MARIANO DE CAVIA,MARIANO DE CAVIA,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
3,31001334,AUTOVÍA,NaN,A-4,A-4,31001349,AUTOVÍA,NaN,M-30,M-30,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
4,31001341,AUTOVÍA,NaN,A-42,A-42,468400,AVENIDA,DEL,MANZANARES,MANZANARES,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL


Una vez seleccionadas las columnas de interés, crearemos una columna adicional que sea la concatenación de la partícula y el nombre de la vía, ya que no estamos interesados de que nuestro modelo detecte las partículas de forma distinta al nombre de la vía.

Así mismo, eliminaremos los registros que contengan vacíos.

In [5]:
df_calles = df_callejero[df_callejero.VIA_CLASE != 'AUTOVÍA'][['VIA_CLASE', 'VIA_PAR', 'VIA_NOMBRE']]

df_calles['NOMBRE_VIA'] = df_calles['VIA_PAR'] + ' ' + df_calles['VIA_NOMBRE']
df_calles.drop(columns = ['VIA_PAR', 'VIA_NOMBRE'], inplace  = True)

df_calles.rename(columns={'VIA_CLASE': 'TIPO_VIA'}, inplace = True)
df_calles.dropna(inplace =  True)

El resultado es una tabla con dos columnas, el tipo de vía y el nombre de la vía.

In [6]:
df_calles.head(5)

,TIPO_VIA,NOMBRE_VIA
7,CALLE,DEL ABAD JUAN CATALAN
8,CALLE,DE LA ABADA
9,CALLE,DE LOS ABADES
10,CALLE,DE LA ABADESA
11,CALLE,DE ABALOS


### 1.2. Generar direcciones ficticias

Para terminar de crear las direcciones ficticias, añadiremos más señas a las calles de Madrid. Para ello, de manera aleatoria, incluiremos información sobre el número y la combinación piso-letra. Este paso lo podemos ir mejorando a medida que descubramos casos reales que no están contemplados en estos que generamos.

In [7]:
def generar_datos_entrenamiento(df, n):
    """
    Genera N líneas aleatorias a partir de un DataFrame base.
    
    Args:
        df (pd.DataFrame): DataFrame base para seleccionar líneas.
        n (int): Número de líneas aleatorias a generar.
    
    Returns:
        pd.DataFrame: DataFrame con N líneas aleatorias generadas.
    """
    lineas = []
    for _ in range(n):
        # Seleccionar una línea aleatoria del DataFrame base
        row = df.sample(1).copy()

        # Agregar un número aleatorio
        pre_numero = random.choice(['n*', 'nr', 'nº', '', 'n'])
        numero = random.randint(1, 50)
        post_numero = random.choice(['', '', '', ','])
        row['NUMERO'] = f'{pre_numero}{numero}{post_numero}'

        # Agregar combinación aleatoria de piso y letra
        piso = random.randint(1, 10)
        letra = random.choice(['A', 'B', 'C', 'D', 'IZDA', 'DCHA', 'CTRO'])
        espacio = random.choice([' ', '*', 'º', '', '-'])
        combinacion = f"{piso}{espacio}{letra}"
        row['RESTO'] = combinacion

        # Agregar la línea generada a la lista
        lineas.append(row)

    # Combinar todas las líneas en un nuevo DataFrame
    return pd.concat(lineas, ignore_index=True)


In [8]:
train_set = generar_datos_entrenamiento(df_calles, 7000)

### 1.3. Estructurar los datos para el modelo

Una vez terminamos de generar las direcciones ficticias, daremos forma a nuestros datos para poder entrenar el modelo de spacy. Esta estructura es específica para la detección de entidades con la librería SpaCy.

Se trata de un tuple en el que encontraremos el texto completo en la primera posición y la definición de las entidades en la segunda. Aquí un ejemplo:

```
('CALLE DE LOS MARTIRES DE PARACUELLOS n44 3 D',
{'entities': [(0, 5, 'TIPO_VIA'),
    (6, 36, 'NOMBRE_VIA'),
    (37, 40, 'NUMERO'),
    (41, 44, 'RESTO')]})
```

En nuestro caso el modelo tendrá 4 tipo de entidades: el tipo de vía, el nombre de la vía, el número y el resto de la dirección. Esto provoca que todo el texto esté identificado con alguna entidad. 

Sin embargo, también podríamos crear modelos que detecten, por ejemplo, únicamente el nombre de la vía. De esta manera no todo el texto estaría categorizado, sino solo una parte.



In [9]:
def crear_entidades(row):
    texto = row['texto']
    entidades = []
    start = 0
    for col, label in zip(['TIPO_VIA', 'NOMBRE_VIA', 'NUMERO', 'RESTO'], ['TIPO_VIA', 'NOMBRE_VIA', 'NUMERO', 'RESTO']):
        value = str(row[col])
        start = texto.find(value, start)  # Buscar el índice inicial de la entidad
        if start != -1:
            end = start + len(value)  # Índice final de la entidad
            entidades.append((start, end, label))
            start = end  # Actualizar el inicio para evitar errores en textos repetidos
    return (texto, {'entities': entidades})

In [10]:
train_set['texto'] = train_set['TIPO_VIA'] + ' ' + train_set['NOMBRE_VIA'] + ' ' + train_set['NUMERO'].astype(str) + ' ' + train_set['RESTO']
train_set['entidades'] = train_set.apply(crear_entidades, axis=1)

lista = train_set['entidades'].to_list()

train, test = train_test_split(lista, test_size=0.25, random_state=8)

## 2. Configurar el modelo

Para entrenar el modelo, primeramente necesitamos seleccionar qué componentes del pipeline del modelo pre-configurado queremos modificar con nuestros datos de entrenamiento. En nuestro caso serán: ner, trf_wordpiecer y trf_tok2vec. 

Los dos últimos son componentes que se encargan de la tokenización y la creación de vectores. Su modificación ayudará al componente ner a mejorar su capacidad de generalización.



In [11]:
nlp = spacy.load('en_core_web_sm')

ner = nlp.get_pipe('ner')

for _,annotations in train:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

pipe_exceptions = ['ner', 'trf_wordpiecer', 'trf_tok2vec']
unaffected_pipe = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

## 3. Entrenar el modelo

Para entrenar el modelo, deshabilitaremos los componentes del pipeline que previamente hemos indicado que no queremos modificar. Después creamos un bucle de entrenamiento con sus respectivos batches (lotes), en el que actualizaremos el modelo y que calcularemos los losses a cada paso del batch. Realizaremos esto tantas veces como iteraciones indiquemos.

Cabe destacar el parámetro "drop", que utilizaremos para descartar ejemplos de entrenamiento y así evitar el overfitting (sobreajuste).

In [12]:
nr_iter = 50

with nlp.disable_pipes(*unaffected_pipe):
    for iteration in range(nr_iter):
        random.shuffle(train)
        losses = {}

        batches = minibatch(train, size=8)
        for batch in batches:
            example = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example.append(Example.from_dict(doc, annotations))

            nlp.update(example, drop = 0.3, losses=losses)

        if iteration % 10 == 0:
            print(f"Iteration {iteration} - Losses: {losses}")


Iteration 0 - Losses: {'ner': np.float32(2569.1147)}
Iteration 1 - Losses: {'ner': np.float32(6.327425)}
Iteration 2 - Losses: {'ner': np.float32(3.6257806)}
Iteration 3 - Losses: {'ner': np.float32(4.602195)}
Iteration 4 - Losses: {'ner': np.float32(13.887782)}
Iteration 5 - Losses: {'ner': np.float32(4.488487)}
Iteration 6 - Losses: {'ner': np.float32(5.294967)}
Iteration 7 - Losses: {'ner': np.float32(9.12825)}
Iteration 8 - Losses: {'ner': np.float32(13.653158)}
Iteration 9 - Losses: {'ner': np.float32(1.9419212)}
Iteration 10 - Losses: {'ner': np.float32(1.6027925e-05)}
Iteration 11 - Losses: {'ner': np.float32(7.8644604e-07)}
Iteration 12 - Losses: {'ner': np.float32(2.0304881e-07)}
Iteration 13 - Losses: {'ner': np.float32(3.8328807e-09)}
Iteration 14 - Losses: {'ner': np.float32(3.9463973e-07)}
Iteration 15 - Losses: {'ner': np.float32(24.40153)}
Iteration 16 - Losses: {'ner': np.float32(2.0032232)}
Iteration 17 - Losses: {'ner': np.float32(0.34888506)}
Iteration 18 - Losses: {

In [16]:
nlp = spacy.load('../models/address_ner')

## 4. Evaluar el modelo



En esta sección, evaluaremos el rendimiento del modelo entrenado utilizando el conjunto de datos de prueba. La evaluación se realizará mediante métricas estándar como precisión, exhaustividad y F1-score. Estas métricas nos permitirán entender cómo el modelo es capaz de identificar y clasificar las entidades nombradas en textos no vistos durante el entrenamiento.

In [17]:
examples = []
scorer = Scorer()
for text, annotations in test:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    example.predicted = nlp(str(example.predicted))
    examples.append(example)

results_scorer = scorer.score(examples)
results_scorer['ents_per_type']

{'TIPO_VIA': {'p': 0.9988571428571429,
  'r': 0.9988571428571429,
  'f': 0.9988571428571429},
 'NOMBRE_VIA': {'p': 0.9942954934398175, 'r': 0.996, 'f': 0.9951470168427062},
 'NUMERO': {'p': 0.9988584474885844, 'r': 1.0, 'f': 0.9994288977727013},
 'RESTO': {'p': 0.9994285714285714,
  'r': 0.9994285714285714,
  'f': 0.9994285714285714}}